In [1]:
!pip install python-mnist

In [1]:
#export
from exp.nb_01 import *
from fastai.vision.all import *
from mnist import MNIST
import pickle
import numpy as np

C:\ProgramData\Anaconda3\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
#export

def fetch(url):
    import requests, os, hashlib, tempfile
    fp = os.path.join(tempfile.gettempdir(), hashlib.md5(url.encode('utf-8')).hexdigest())
    if os.path.isfile(fp) and os.stat(fp).st_size > 0:
        with open(fp, "rb") as f:
            dat = f.read()
    else:
        print("fetching %s" % url)
        dat = requests.get(url).content
        with open(fp+".tmp", "wb") as f:
            f.write(dat)
        os.rename(fp+".tmp", fp)
    return dat

In [16]:
#export
def get_data():
    import gzip
    parse = lambda dat: np.frombuffer(gzip.decompress(dat), dtype=np.uint8).copy()
    X_train = tensor(parse(fetch("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz"))[0x10:]).reshape((-1, 28*28)).float()/255
    Y_train = tensor(parse(fetch("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz"))[8:]).long()
    X_test = tensor(parse(fetch("http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz"))[0x10:]).reshape((-1, 28*28)).float()/255
    Y_test = tensor(parse(fetch("http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz"))[8:]).long()
    return (X_train,Y_train,X_test,Y_test)

In [8]:
#export
def normalize(x, m, s): return (x-m)/s

In [9]:
x_train,y_train,x_valid,y_valid = get_data()

In [10]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1307), tensor(0.3081))

In [11]:
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [12]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(-7.0585e-09), tensor(1.0000))

In [13]:
#export
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

In [14]:

test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [15]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(60000, 784, tensor(10, dtype=torch.uint8))

In [16]:
# num hidden
nh = 50

In [17]:
# standard xavier init
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [18]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [19]:
# This should be ~ (0,1) (mean,std)...
x_valid.mean(),x_valid.std()

(tensor(0.0060), tensor(1.0077))

In [20]:
def lin(x, w, b): return x@w + b

In [21]:

t = lin(x_valid, w1, b1)

In [22]:
#...so should this, because we used xavier init, which is designed to do this
t.mean(),t.std()

(tensor(0.0193), tensor(0.9671))

In [23]:
def relu(x): return x.clamp_min(0.)

In [24]:
t = relu(lin(x_valid, w1, b1))

In [25]:
#...actually it really should be this!
t.mean(),t.std()

(tensor(0.3866), tensor(0.5800))

In [26]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2/m)

In [27]:
w1.mean(),w1.std()

(tensor(0.0003), tensor(0.0502))

In [28]:
t = relu(lin(x_valid, w1, b1))
t.mean(),t.std()

(tensor(0.4831), tensor(0.7725))

In [29]:
#export
from torch.nn import init

In [30]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))

In [31]:
w1.std()

tensor(0.0504)

In [32]:
import torch.nn

In [33]:
def relu(x): return x.clamp_min(0.) - 0.5

In [34]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2./m )
t1 = relu(lin(x_valid, w1, b1))
t1.mean(),t1.std()

(tensor(-0.0127), tensor(0.7763))

In [35]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [36]:
#export
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

In [37]:

y_train,y_valid = y_train.float(),y_valid.float()

In [38]:
preds = model(x_train)

In [39]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [40]:
#export
from torch import nn

In [15]:
!python notebook2script.py 02_fully_connected.ipynb

Converted 02_fully_connected.ipynb to exp\nb_02.py
